# Classification binaire: prédire la détention du baccalauréat dans le recensement 

## 1. Présentation
Dans ce tutoriel, nous allons voir comment utiliser une forêt aléatoire en Python avec l'implémentation de `scikit-learn`. Cet algorithme est présenté en détail dans le document de travail ([ici](https://inseefrlab.github.io/DT_methodes_ensemblistes/chapters/chapter2/3-random_forest.html)).

Ce tutoriel se concentre donc sur la mise en oeuvre pratique de l'algorithme. Nous présentons les étapes suivantes: la préparation des données, la construction du modèle, l'entraînement du modèle, l'optimisation des hyperparamètres et quelques éléments d'interprétation des résultats.

Le jeu de données est issu du recensement de la population (Insee) et contient des informations individuelles issues du recensement, telles que l'âge, le niveau d'éducation, la situation professionnelle, etc. L'objectif sera de prédire si un individu a obtenu le baccalauréat en fonction de ses autres caractéristiques observées.


## 2. Préparation de l'environnement

### 2.1 Importation des bibliothèques nécessaires

In [ ]:
# Importation des bibliothèques pour la manipulation des données
import os
import s3fs
import zipfile
import pandas as pd
import pyarrow.parquet as pq
import numpy as np
import copy

# Bibliothèques pour la visualisation
import matplotlib.pyplot as plt
import seaborn as sns


# Bibliothèques pour le traitement des données
import sklearn
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.pipeline import Pipeline

# Bibliothèques pour le modèle et l'évaluation
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, ParameterGrid
from sklearn.metrics import (
    confusion_matrix,
    classification_report,
    ConfusionMatrixDisplay, 
    roc_curve,
    RocCurveDisplay,
    brier_score_loss,
    log_loss,
    roc_auc_score
)

# Pour mesurer le temps d'exécution
import time

### 2.2 Chargement des données

In [ ]:
# Définir la liste des variables à conserver
variable_list = [
    'AGED', 'APAF', 'CATL', 'CATPC', 'COUPLE', 'CS1', 'DEPT', 'DIPL', 
    'EMPL', 'HLML', 'ILETUD', 'ILT', 'IMMI', 'INAI', 'INATC', 'INFAM', 
    'INPER', 'INPERF', 'IRAN', 'LIENF', 'LPRF', 'LPRM', 'METRODOM', 'MOCO',
     'MODV', 'NA17', 'NA5', 'NAIDT', 'NBPI', 'NENFR', 'NPERR', 'ORIDT', 'SEXE', 
     'SFM', 'STAT_CONJ', 'STATR', 'STOCD', 'SURF', 'TACTD16', 
     'TP', 'TRANS', 'TYPC', 'TYPFC'
]

In [ ]:
# Se connecter au bucket
endpoint = "https://"+os.environ['AWS_S3_ENDPOINT']
fs = s3fs.S3FileSystem(client_kwargs={'endpoint_url': endpoint}, anon=True)

# Charger les données individuelles  du recensement
with fs.open('s3://oliviermeslin/rp/data_census_individuals.parquet', 'rb') as file:
    data_census_individuals = pd.read_parquet(file, columns = variable_list)

# Charger la documentation
with fs.open('s3://oliviermeslin/rp/metadata_census_individuals.csv', 'rb') as file:
    doc_census_individuals = pd.read_csv(file, sep=';')

## 3. Préparation des données

### 3.1 Echantillonnage des données
Compte tenu de l'objectif pédagogique de ce tutoriel, nous tirons un échantillon aléatoire de taille restreinte (0,2 % des observations), représentatif de l'ensemble des données initiales, afin d'accélérer les calculs dans les sections suivantes.

In [ ]:
# Échantillonner les données (1/200)
data_sample = data_census_individuals.sample(frac=1/500, random_state=123)

### 3.2 Création de la variable cible

Nous créons la variable que l'on va tâcher de prédire: une variable indicatrice qui vaut 1 pour les individus détenteurs du baccalauréat, 0 pour les autres. Un point important est que les deux classes sont approximativement équilibrées, il n'est donc pas nécessaire d'utiliser des pondérations.

In [ ]:
# Voir la répartition des classes de diplôme
print(data_sample['DIPL'].isna().sum())  # Vérification des valeurs manquantes
print(data_sample['DIPL'].value_counts())  # Distribution des classes

In [ ]:
# Préciser l'odre des catégories de DIPL (ZZ suivi des valeurs numériques croissantes)
categories = ['ZZ'] + [f"{i:02}" for i in range(1, 20)]
data_sample['DIPL'] = pd.Categorical(data_sample['DIPL'], categories=categories, ordered=True)

# Créer la variable binaire 'bac'
data_sample['bac'] = (data_sample['DIPL'] > '13').astype(int)

# Afficher le résultat: les deux classes sont approximativement équilibrées
print(data_sample['bac'].value_counts())  # Distribution des classes

# Supprimer la colonne 'DIPL' du DataFrame (car c'est un prédicteur parfait du baccalauréat)
data_sample = data_sample.drop(columns=['DIPL'])

### 3.3 Préparation des variables explicatives (_features_)
Dans un premier temps, on transforme en variables continues toutes les variables catégorielles qui peuvent l'être. Certaines variables catégorielles comportent un ordre naturel, mais comprennent également une modalité "X" ou "Z" correspondant à une valeur manquante (ou non pertinente). Nous remplaçons cette modalité par une valeur numérique extrême (99 ou 999), ce qui permet de convertir la variable en variable continue.

In [ ]:
# Dupliquer la table de données avant de retraiter les features
data_clean = copy.deepcopy(data_sample)

In [ ]:
# Créer une fonction qui teste si une chaîne de caractères peut être convertie en un entier
def can_convert_to_int(value):
    try:
        int(value)  # Attempt to convert the value to an integer
        return True
    except ValueError:
        return False

In [ ]:
# Transformer en variables continues toutes les variables qui peuvent l'être
for var in data_clean.columns.tolist():
    if data_clean[var].dtype.name == "category":
        if set([category for category in data_clean[var].cat.categories if not can_convert_to_int(category)]) <= set(['X', 'Z', 'ZZ']):
            data_clean[var] = data_clean[var].cat.rename_categories({'X': '99', 'Z': '999', 'ZZ': '9999'})
            data_clean[var] = data_clean[var].astype(int)

On sépare enfin les _features_ et la variable-cible.

In [ ]:
# Séparer les features (X) et la variable cible (y : bac)
X = data_clean2.drop(columns=['bac'])
y = data_clean['bac']

### 3.4 Séparation des ensembles d'entraînement et de test
On utilise la fonction [`train_test_split()`](https://scikit-learn.org/dev/modules/generated/sklearn.model_selection.train_test_split.html) pour séparer l'ensemble d'entraînement et l'ensemble de test. La stratification en fonction de $y$ assure que la proportion des classes est la même dans les ensembles d'entraînement et de test. On rappelle qu'un ensemble de test n'est pas absolument nécessaire pour évaluer une forêt aléatoire.

In [ ]:
# Diviser les données en ensembles d'entraînement (80%) et de test (20%) avec stratification
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=123, stratify=y
)

## 4. Construire le modèle complet

Nous allons construire le modèle complet en utilisant __un _pipeline_ `scikit-learn`, dont l'usage est très fortement recommandé__. Ce _pipeline_ va comprendre deux étapes: le _preprocessing_ des données, puis la forêt aléatoire proprement dite.

Un _pipeline_ `scikit-learn` est un objet informatique qui permet d'assurer une exécution cohérente de plusieurs étapes de traitement des données. Par exemple, un _pipeline_ permet de réaliser la normalisation des variables, le traitement des valeurs manquantes, l'encodage des variables catégorielles, et l'entraînement d'un modèle __en une seule étape__. Utiliser un _pipeline_ simplifie les codes et garantit que les mêmes transformations sont appliquées aux données d'entraînement et de test, ce qui réduit le risque d'erreur et augmente la reproductibilité des résultats. Si vous n'êtes pas habitué à les utiliser, il est fortement recommandé de lire la [documentation officielle sur ce sujet](https://scikit-learn.org/1.5/modules/compose.html) et l'[excellente formation](https://pythonds.linogaliana.fr/content/modelisation/6_pipeline.html) de Lino Galiana.

### 4.1 Le préprocessing des données
Le _préprocessing_ des données est construit avec la fonction  [`ColumnTransformer()`](https://scikit-learn.org/1.5/modules/generated/sklearn.compose.ColumnTransformer.html) de `scikit` et n'est pas le même selon le type des variables:
- on applique un _one-hot-encoding_ aux variables catégorielles, avec la fonction [`OneHotEncoder()`](https://scikit-learn.org/dev/modules/generated/sklearn.preprocessing.OneHotEncoder.html) de `scikit`. On choisit d'encoder toutes les modalités (`drop=None`). L'option `handle_unknown='error'` signifie que le modèle renverra une erreur s'il rencontre une modalité inconnue lors de l'utilisation en prédiction.
- On applique aucune transformation aux variables numériques car les méthodes ensemblistes sont invariantes aux transformations monotones des _features_. On utilise donc l'option "passthrough".

On utilise la fonction [`make_column_selector()`](https://scikit-learn.org/1.5/modules/generated/sklearn.compose.make_column_selector.html) permet d'appliquer le bon _preprocessing_ en fonction du type des variables, sans avoir à en faire une liste explicite.

Pour plus d'explications, vous pouvez lire la partie du document de travail sur la [préparation des données](https://inseefrlab.github.io/DT_methodes_ensemblistes/chapters/chapter3/1-preparation_donnees.html).

In [ ]:
# Construire la première étape du pipeline (préprocessing des données)

# One-hot-encoding des variables catégorielles
preprocessor  = ColumnTransformer( 
    [
        (
            'encoder', 
            OneHotEncoder(drop = None, handle_unknown='ignore'), 
            make_column_selector(dtype_include=["category", "object"])
        )
    ], 
    remainder='passthrough', 
    verbose_feature_names_out=False,
    sparse_threshold = 0
)

### 4.2 Définir le _pipeline_

On définit ensuite la seconde étape du traitement: le modèle de _machine learning_ proprement dit. En l'occurrence, il s'agit d'une forêt aléatoire.

In [ ]:
# Construire la seconde étape du pipeline (la forêt aléatoire)
rf_algorithm = RandomForestClassifier(random_state=42, n_jobs=-1)

Une fois que les étapes de modélisation sont définies, on peut les rassembler explicitement dans un _pipeline_ pour constituer le modèle complet. C'est ce modèle que nous allons utiliser tout au long de ce tutoriel.

In [ ]:
# Construire le pipeline complet
rf_model = Pipeline(
    steps=[
        # Étape 1: preprocessing
        ("preprocessor", preprocessor),
        # Étape 2: le modèle de ML
        ("classifier", rf_algorithm)
    ]
)

### 4.3 Automatiser la construction du modèle
Maintenant que nous avons vu comment construire un modèle complet sous forme de _pipeline_, nous allons automatiser cette tâche sous la forme d'une fonction. Cela permettra de générer un nouveau modèle à chaque fois que nous en aurons besoin. Ci-dessous, on définit la fonction `create_model()` qui reproduit le code présenté précédemment, et renvoie le modèle complet.

In [ ]:
def create_model(**params):
    # One-hot-encoding des variables catégorielles
    preprocessor  = ColumnTransformer( 
        [
            (
                'encoder', 
                OneHotEncoder(drop = None, handle_unknown='error'), 
                make_column_selector(dtype_include=["category", "object"])
            )
        ], 
        remainder='passthrough', 
        verbose_feature_names_out=False,
        sparse_threshold = 0
    )

    # Construire la seconde étape du pipeline (la forêt aléatoire)
    rf_algorithm = RandomForestClassifier(**params)
    
    # Construire le pipeline complet
    rf_model = Pipeline(
        steps=[
            # Etape 1: preprocessing
            ("preprocessor", preprocessor),
            # Etape 2: le modèle de ML
            ("classifier", rf_algorithm)
        ]
    )
    return(rf_model)

## 5. Entraîner une première forêt aléatoire 
Dans cette section, nous allons entraîner une toute première forêt avec les __hyperparamètres par défaut__. Il est préférable d'avoir lu la [section](https://inseefrlab.github.io/DT_methodes_ensemblistes/chapters/chapter3/2-guide_usage_RF.html#sec-hyperparam-rf) du document de travail sur les hyperparamètres des forêts aléatoires avant de commencer cette partie.

Il est important de noter que ces valeurs par défaut varie d'une implémentation à l'autre. Dans `scikit-learn`, [`RandomForestClassifier`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html#sklearn.ensemble.RandomForestClassifier) a les valeurs par défaut suivantes:

|Hyperparamètre | Signification | Valeur par défaut |
|:----:|----|:----:|
| `n_estimators`           | Nombre d'arbres dans la forêt | 100 |
| `max_features`           | Nombre de variables à considérer pour déterminer le meilleur _split_ | 'sqrt'|
| `max_samples`            | La taille des échantillons aléatoires                                         | `n` |
| `min_samples_leaf`       | Nombre minimum d'échantillons dans une feuille terminale | 1 |
| `min_samples_split`      | Nombre minimal d'observations nécessaire pour qu'un noeud puisse être partagé | 2 |
| `criterion`              | Le critère de choix de la règle de division des noeuds intermédiaires         | 'gini' |
| `max_depth`              | Profondeur maximale des arbres                                                | None |

### 4.1 Entraînement du modèle

On commence par créer un modèle en faisant appel à la fonction `RandomForestClassifier()`.

In [ ]:
# Créer un modèle
rf_default_settings = create_model()

In [ ]:
# Entraînement du modèle
start_time = time.time()
rf_default_settings.fit(X_train, y_train)
elapsed_time = time.time() - start_time
print(f"Temps d'entraînement du modèle Random Forest : {elapsed_time:.2f} secondes")

### 4.2 Représenter les arbres

Le code ci-dessous permet de représenter un arbre. Attention, il est indispensable de spécifier `max_depth`, sinon le graphique sera ilisible!

In [ ]:
import matplotlib.pyplot as plt
from sklearn.tree import plot_tree
# Plot the tree using the plot_tree function from sklearn
tree = rf_default_settings["classifier"].estimators_[0]
plt.figure(figsize=(20,10))  # Set figure size to make the tree more readable
plot_tree(tree, 
          filled=True,              # Fill nodes with colors for better visualization
          rounded=True,
          max_depth=2)             # Rounded edges for nodes
plt.title("Decision Tree from the Random Forest")
plt.show()

### 4.3 Evaluation sur le jeu de test

La matrice de confusion permet de visualiser les erreurs de classification.
Le rapport de classification donne des métriques importantes comme la précision, le rappel et le F1-score.

In [ ]:
# Prédictions sur les données de test
y_pred_baseline = rf_default_settings.predict(X_test)

In [ ]:
# Évaluer la performance avec une matrice de confusion
conf_matrix = confusion_matrix(y_test, y_pred_baseline, labels=rf_default_settings.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=conf_matrix, display_labels=rf_default_settings.classes_)
disp.plot(cmap="Blues")
plt.title("Matrice de confusion pour la prédiction de la détention du baccalauréat")
plt.show()

In [ ]:
# Rapport de classification
print(classification_report(y_test, y_pred_baseline))

In [ ]:
# Représenter la ROC curve
RocCurveDisplay.from_estimator(model_default_settings, X_test, y_test, name="Model with default settings")
plt.show()

## 5. Optimisation des hyperparamètres

Dans cette section, nous allons suivre pas à pas la procédure d'entraînement des forêts aléatoires détaillée [ici](https://inseefrlab.github.io/DT_methodes_ensemblistes/chapters/chapter3/2-guide_usage_RF.html). Nous allons donc aborder les étapes suivantes

- Choisir le bon nombre d'arbres (`n_estimators`);
- Explorer l'espaces des valeurs possibles pour min_samples_leaf avec un grid search (par validation croisée et par l'approche OOB);
- Explorer l'espaces des valeurs possibles pour `max_features` (alias de `mtry`) et `max_samples` avec un _grid search_;
- Evaluer l'apport de cette optimisation des hyperparamètres en comparant le modèle final à un modèle reposant sur les valeurs par défaut des hyperparamètres.

### 5.1 Choisir le nombre d'arbres (`n_estimators`)

Le nombre d'arbres d'une forêt aléatoire est un hyperparamètre particulier car il n'est associé à aucun arbitrage en matière de performance: la performance de la forêt aléatoire croît avec le nombre d'arbres, puis se stabilise à un niveau approximativement constant. Le nombre optimal d'arbres est donc intuitivement celui à partir duquel la performance de la forêt ne croît plus: il en faut "assez", mais pas "trop", d'autant qu'un nombre d'arbre élevé peut allonger considérablement le temps de calcul.

La méthode proposée pour choisir le nombre d'arbre est la suivante:

- on entraîne une forêt aléatoire avec les hyperparamètres par défaut, en augmentant progressivement le nombre d'arbres;
- on calcule à chaque étape le taux d'erreur _out-of-bag_ du modèle en fonction du nombre d'arbres, en utilisant le [score de Brier](https://scikit-learn.org/dev/modules/model_evaluation.html#common-cases-predefined-values) comme métrique;
- on représente graphiquement ce taux d'erreur en fonction du nombre d'arbres;
- le nombre d'arbres optimal est celui à partir duquel le taux d'erreur ne diminue plus.

Deux remarques sur cette approche:

- Cette approche n'est pas parfaite, car le nombre optimal d'arbres dépend de la valeur des autres hyperparamètres, mais elle a le mérite d'être simple et rapide. 
- Il est possible d'appliquer cette approche avec une autre métrique que le score de Brier. La liste des métriques disponibles dans `scikit-learn` est [ici](https://scikit-learn.org/dev/modules/model_evaluation.html#common-cases-predefined-values).

In [ ]:
# Ce code est inspiré de celui-ci: https://scikit-learn.org/stable/auto_examples/ensemble/plot_ensemble_oob.html

# Créer un modèle avec les hyperparamètres par défaut
# NOTE: Setting the `warm_start` construction parameter to `True` disables
# support for parallelized ensembles but is necessary for tracking the OOB
# error trajectory during training.
rf_nb_trees = RandomForestClassifier(
    oob_score=brier_score_loss, # Ici il est possible de choisir une autre métrique pour calculer l'erreur OOB
    warm_start=True,
    random_state=42, 
    n_jobs=-1
)

# Préparer un dictionnaire pour stocker les résultats
error_rate = dict()

# Définir l'intervalle de valeurs à explorer
min_estimators = 40
max_estimators = 500
increment_size = 20

# Entraîner le modèle en augmentant le nombe d'arbres
for n_trees in range(min_estimators, max_estimators + 1, increment_size):
    print(n_trees)

    # Définir le nombre d'arbres
    rf_nb_trees.set_params(n_estimators=n_trees)
    rf_nb_trees.fit(X_train, y_train)

    # Conserver le taux d'erreur du modèle
    error_rate[n_trees] = rf_nb_trees.oob_score_

In [ ]:
# Représenter graphiquement le taux d'erreur en fonction du nombre d'arbres
x, y = zip(*sorted(error_rate.items()))
plt.plot(x, y)
plt.xlim(min_estimators, max_estimators)
plt.xlabel("Nombre d'arbres")
plt.ylabel(f"Taux d'erreur OOB (métrique = {rf_nb_trees.oob_score.__name__})")
plt.show()

On voit que la performance croît nettement avec le nombre d'arbres au début, puis se stabilise progressivement. La performance s'améliore peu au-delà de 200 arbres: on retient donc la valeur de 200 arbres.

### 6.2 Choisir la taille de l'échantillon (`max_samples`) et la proportion de variables candidates (`max_features`)

Nous allons maintenant choisir le taux d'échantillonnage et la proportion de variables candidates. Pour ce faire, nous allons explorer l'espace des valeurs possibles pour ces hyperparamètres, selon deux approches qui donnent généralement des résultats similaires (mais pas forcément identiques):

- Une approche par validation croisée avec la fonction `GridSearchCV`;
- Une approche par l'erreur OOB (_out-of-bag_).

Deux points importants sont à noter:

- l'approche par validation croisée est intense en calcul, mais est complètement standard et applicable à tous les algorithmes de machine learning supervisé. Inversement, __l'approche par l'ereur OOB est plus rapide, mais elle est exclusivement applicable aux forêts aléatoires__ et ne peut pas être utilisée avec d'autres algorithmes. Par ailleurs, elle est beaucoup moins bien (voire pas) documentée sur internet.
- si vous testez un grand nombre de valeurs possibles pour les hyperparamètres par validation croisée, le temps de calcul peut devenir très long!

Nous utilisons un modèle avec les hyperparamètres choisis jusqu'ici: `n_estimators = 200`. On rajoute également `min_samples_leaf = 10` car cela accélère l'entraînement, le plus souvent sans effet notable sur les performances.

In [ ]:
# APPROCHE 1: Validation croisée

# Créer un modèle avec 200 arbres
model_grid_search = create_model(n_estimators = 200, min_samples_leaf = 10)

# Créer un modèle avec 400 arbres, et tous les autres hyperparamètres par défaut
rf_crossval = RandomForestClassifier(
    n_estimators = 400,
    random_state=42, 
    n_jobs=-1
)

# Définir la grille d'hyperparamètres
param_grid = {
    # Taille de l'échantillon
    'classifier__max_samples': [0.6, 0.8, 1.0],
    # Nombre de features candidates à chaque split 
    'max_features': ['sqrt', 0.6, 0.8, None] # None revient à faire du bagging
}

# Configurer GridSearchCV
grid_search = GridSearchCV(
    estimator=rf_crossval,
    param_grid=param_grid,
    cv=5,                         # Validation croisée à 5 plis
    scoring='neg_brier_score',    # Métrique à optimiser
    n_jobs=-1,                    # Utiliser tous les cœurs disponibles
    verbose=1,                    # Afficher la progression
    refit=True                    # Réentrainer le modèle avec les meilleurs paramètres
)

# Exécuter la recherche
grid_search.fit(X_train, y_train)

# Afficher les meilleurs hyperparamètres
print("Best parameters found:", grid_search.best_params_)

In [ ]:
# APPROCHE 2: approche OOB
# ATTENTION: cette approche est applicable exclusivement aux forêts aléatoires

# Créer une variable pour conserver le score
best_score = 0

# Définir la grille d'hyperparamètres
param_grid = {
    # Taux d'échantillonnage
    'max_samples': [0.6, 0.8, 1.0],
    # Nombre de features candidates à chaque split 
    'max_features': ['sqrt', 'log2', None] # None revient à faire du bagging
}

# Entraîner le modèle et calculer l'erreur OOB avec la métrique choisie
for g in ParameterGrid(param_grid):

    # Créer un modèle avec 200 arbres
    model_oob = create_model(
        n_estimators=200, 
        min_samples_leaf=10,
        oob_score=roc_auc_score, # Ici il est possible de choisir une autre métrique pour calculer l'erreur OOB
        random_state=42, 
        n_jobs=-1,
        **g
    )
    # Entraîner le modèle
    model_oob.fit(X_train, y_train)
    # Conserver le meilleur
    if model_oob["classifier"].oob_score_ > best_score:
        best_score = model_oob["classifier"].oob_score_
        best_params = g
        best_model_oob = copy.deepcopy(model_oob)
        print(f"{best_params}: {best_score}")

print("Meilleurs hyperparamètres:", best_params)

In [ ]:
# Évaluer le meilleur modèle obtenu par l'approche Gridsearch sur les données de test
best_model_gs = grid_search.best_estimator_
y_pred_gs = best_model_gs.predict(X_test)
print("\nClassification Report:\n", classification_report(y_test, y_pred_gs))

In [ ]:
# Évaluer le meilleur modèle obtenu par l'approche OOB sur les données de test
y_pred_oob = best_model_oob.predict(X_test)
print("\nClassification Report:\n", classification_report(y_test, y_pred_oob))

### 6.3 Influence de la taille minimale des feuilles (_min_samples_leaf_)

Pour de faibles valeurs du nombre minimum d'observations par feuille (noeud terminal), le modèle risque de sur-ajuster les données d'entraînement; pour des valeurs élevées, le modèle devient plus simple, ce qui peut entraîner un sous-ajustement. Cet hyperparamètre n'est pas le plus important en termes de performance, mais il a une __influence majeure sur le temps d'entraînement du modèle__: une valeur faible implique des arbres profonds donc longs à entraîner. Il peut donc parfois être utile de vérifier assez tôt dans la procédure d'entraînement s'il est possible de le fixer à une valeur plus élevée que la valeur par défaut sans perte de performance, pour accélérer le reste de la procédure. Ceci dit, il faut garder en tête que la valeur optimale de cet hyperparamètre peut dépendre des autres hyperparamètres. 

Le code suivant montre comment on peut rapidement définir une valeur raisonnable pour cet hyperparamètre. Les résultats indiquent qu'une valeur de 10 est raisonnable.


In [ ]:
# APPROCHE 1: Validation croisée

# Créer un modèle avec 400 arbres, max_features = 0.6 et max_samples = 0.7
rf_crossval_leaf_size = RandomForestClassifier(
    n_estimators = 400,
    max_features = 0.6,
    max_samples = 0.5,
    random_state=42, 
    n_jobs=-1
)

# Définir la grille d'hyperparamètres
param_grid_leaf_size = {
    # Taille minimale des feuilles
    'min_samples_leaf': [1, 3, 5, 10, 20, 50, 100, 200, 500]
}

# Configurer GridSearchCV
grid_search_leaf_size = GridSearchCV(
    estimator=rf_crossval_leaf_size,
    param_grid=param_grid_leaf_size ,
    cv=5,                         # Validation croisée à 5 plis
    scoring='neg_brier_score',    # Métrique à optimiser
    n_jobs=-1,                    # Utiliser tous les cœurs disponibles
    verbose=1,                    # Afficher la progression
    refit=True                    # Réentrainer le modèle avec les meilleurs paramètres
)

# Exécuter la recherche
grid_search_leaf_size.fit(X_train, y_train)

# Afficher les meilleurs hyperparamètres
print("Best parameters found:", grid_search_leaf_size.best_params_)

In [ ]:
# APPROCHE 2: approche OOB
# ATTENTION: cette approche est applicable exclusivement aux forêts aléatoires

# Créer une variable pour conserver le score
best_score_leaf_size = np.inf

# Créer un modèle avec 400 arbres, max_features = 0.6 et max_samples = 0.7
rf_oob_leaf_size = RandomForestClassifier(
    n_estimators = 400,
    max_features = 0.6,
    max_samples = 0.5,
    oob_score=brier_score_loss, # Ici il est possible de choisir une autre métrique pour calculer l'erreur OOB
    random_state=42, 
    n_jobs=-1
)

# Définir la grille d'hyperparamètres
param_grid_leaf_size = {
    # Taille minimale des feuilles
    'min_samples_leaf': [1, 3, 5, 10, 20, 50, 100, 200, 500]
}

for g in ParameterGrid(param_grid_leaf_size):
    print(g)
    rf_oob_leaf_size.set_params(**g)
    rf_oob_leaf_size.fit(X_train,y_train)
    print(rf_oob_leaf_size.oob_score_)
    # save if best
    if rf_oob_leaf_size.oob_score_ < best_score:
        best_score = rf_oob_leaf_size.oob_score_
        best_params = g

print("OOB: %0.5f" % best_score) 
print("Best parameters found:", best_params)

Les deux approches donnent à nouveau des résultats différents mais très proches: `min_samples_leaf = 5` et `min_samples_leaf = 3`,  soit des valeurs légèrement plus élevées que la valeur par défaut. On retient `min_samples_leaf = 5`. Le script suivant illustre l'influence de cet hyperparamètre sur la performance du modèle, mesurée par l'aire sous la courbe ROC. On voit bien qu'il y a un arbitrage entre des valeurs faibles (qui aboutissent à des arbres très profonds qui surajustent les données) et des valeurs élevées (qui aboutissent à des arbres trop simples et peu performants).

In [ ]:
# Différentes valeurs de min_samples_leaf à tester
min_samples_leaf_range = [5, 10, 15, 20, 30, 40, 50, 75, 100, 150, 200]
scores = []

for min_samples_leaf in min_samples_leaf_range:
    t1 = time.time()

    print(min_samples_leaf)

    # Créer un modèle avec 200 arbres
    model_oob = create_model(
        n_estimators=200, 
        min_samples_leaf=min_samples_leaf,
        oob_score=roc_auc_score,
        random_state=42, 
        n_jobs=-1,
        **g
    )
    # Entraîner le modèle
    model_oob.fit(X_train,y_train)
    t2 = time.time()
    print(f'Training a model with min_samples_leaf={min_samples_leaf} took {t2-t1:3.1f} secondes.')

    scores.append(model_oob["classifier"].oob_score_)

# Tracer les résultats
plt.plot(min_samples_leaf_range, scores, marker='o')
plt.xlabel('Valeurs de min_samples_leaf')
plt.ylabel('Accuracy')
plt.title('Influence de min_samples_leaf')
plt.grid()
plt.show()

### 6.4 Entraîner le modèle final

A l'issue des différentes étapes d'optimisation des hyperparamètres, nous arrivons au choix suivant:

- `n_estimators = 200`;
- `max_features = None`,
- `max_samples = 10`;
- `min_samples_leaf = 10`.

On entraîne le modèle final avec ces hyperparamètres, et on affiche sa matrice de confusion.

In [ ]:
# Créer le modèle
best_model = create_model(warm_start = True, oob_score = True)

# Définir les hyperparamètres
param_final = {
    # Taille de l'échantillon
    'classifier__n_estimators': 200,
    # Nombre de features candidates à chaque split 
    'classifier__max_features': None,
    # Taux d'échantillonnage des données
    'classifier__max_samples': 0.6,
    # Taille minimale des feuilles terminales
    'classifier__min_samples_leaf': 10
}

# Passer les hyperparamètres au modèle
best_model.set_params(
    **param_final
)

# Entraîner le modèle
best_model.fit(X_train,y_train)

In [ ]:
# Prédictions sur les données de test
y_pred_best = best_model.predict(X_test)

In [ ]:
# Évaluer la performance avec une matrice de confusion
conf_matrix = confusion_matrix(y_test, y_pred_best, labels=best_model.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=conf_matrix, display_labels=best_model.classes_)
disp.plot(cmap="Blues")
plt.title("Matrice de confusion pour la prédiction du niveau de diplôme")
plt.show()

In [ ]:
# Rapport de classification
print(classification_report(y_test, y_pred))

In [ ]:
# Représenter la ROC curve
RocCurveDisplay.from_estimator(best_model, X_test, y_test, name="Best model")
plt.show()

### 6.5 Comparaison avec le modèle de base
On va maintenant comparer les performances du modèle final avec celles du  modèle entraîné avec les hyperparamètres par défaut. Le graphique ci-dessous compare les deux courbes ROC. On voit que le modèle final a des performances légèrement supérieures à celles du modèle de base. Cela illustre deux propriétés importantes des forêts aléatoires: __elles sont relativement peu sensibles aux valeurs des hyperparamètres, et un modèle entraîné avec les valeurs par défaut présente généralement des performances satisfaisantes.__

In [ ]:
ax = plt.gca()  # get current axes

# Plot ROC curve for the first random forest
rfc_disp1 = RocCurveDisplay.from_estimator(model_default_settings, X_test, y_test, ax=ax, curve_kwargs={"alpha": 0.8}, name="Model with default settings")

# Plot ROC curve for the second random forest on the same plot
rfc_disp2 = RocCurveDisplay.from_estimator(best_model, X_test, y_test, ax=ax, curve_kwargs={"alpha": 0.8}, name="Best model")

plt.title("Comparaison des deux modèles")
plt.show()

## 7. Interprétation du modèle : Importance des variables

### 7.1 Extraire l'importance des variables
L'importance des variables reflète leur contribution à la qualité des prédictions du modèle. Parmi les différentes mesures disponibles, nous utilisons ici la "Mean Decrease in Impurity" (MDI). Cette métrique évalue la contribution de chaque variable à la réduction de l'impureté moyenne dans l'arbre de décision. 
Pour chaque variable, elle correspond à la moyenne des réductions d’impureté (par exemple, l'indice de Gini ou l'entropie) qu’elle a engendrées dans tous les nœuds de tous les arbres où elle est impliquée. Les variables présentant la réduction moyenne d’impureté la plus élevée sont considérées comme les prédicteurs les plus importants.

In [ ]:
# Importance des variables (Mean Decrease in Impurity)
importance_df = pd.DataFrame({
    'Variable_modele': best_model["preprocessor"].get_feature_names_out(),
    'Importance': best_model["classifier"].feature_importances_
}).sort_values(by='Importance', ascending=False)

In [ ]:
importance_df

Si on affiche la table d'importance, on constate tout de suite une difficulté: l'importance des variables catégorielles qui ont fait l'objet d'un _one-hot-encoding_ est répartie sur plusieurs lignes (une par variable issue du _one-hot-encoding_). Il faut donc réagréger les mesures d'importance pour obtenir des données plus interprétables. 

In [ ]:
# Construire un dictionnaire qui associe nom de variables et éléments de la table d'importance
dict_ohe = {}
for i in range(len(best_model["preprocessor"]["encoder"].feature_names_in_.tolist())):
    dict_ohe[best_model["preprocessor"]["encoder"].feature_names_in_.tolist()[i]] =  [
        best_model["preprocessor"]["encoder"].feature_names_in_.tolist()[i] + '_' + z 
        for z in best_model["preprocessor"]["encoder"].categories_[i].tolist()
    ]

dict_ohe = {v: k for k, values in dict_ohe.items() for v in values}

dict_remainder = {}
for i in range(len(best_model["preprocessor"]["remainder"].feature_names_in_.tolist())):
    dict_remainder[best_model["preprocessor"]["remainder"].feature_names_in_.tolist()[i]] =  best_model["preprocessor"]["remainder"].feature_names_in_.tolist()[i]

dict_labels = merged = {**dict_ohe, **dict_remainder}

In [ ]:
importance_df["Variable"] = importance_df["Variable_modele"].map(dict_labels)

In [ ]:
# Récupérer les noms des colonnes
feature_names = X_train.columns
raw_feature_names = data_sample.drop("bac", axis = 1).columns

# Appeler la fonction pour regrouper les importances
aggregated_importance_df = importance_df.groupby("Variable")["Importance"].sum().reset_index()

# Afficher les résultats
print(aggregated_importance_df)

In [ ]:
rf_final.feature_importances_

In [ ]:
# Charger les libellés des variables depuis le dictionnaire
doc_census_individuals_noms_variables = doc_census_individuals[['COD_VAR', 'LIB_VAR']].drop_duplicates()

In [ ]:
# Associer les libellés (descriptions) aux variables d'importance
importance_df_with_labels = aggregated_importance_df.merge(
    doc_census_individuals_noms_variables,
    left_on='Variable',
    right_on='COD_VAR',
    how='left'
)

### 7.2 Visualisation des variables les plus importantes
La visualisation aide à comprendre quelles variables contribuent le plus à la prédiction.

In [ ]:
# Fonction pour visualiser les n variables les plus importantes
def plot_top_n_variables(importance_df, variable_col='Variable', importance_col='Importance', top_n=10):
    """
    Visualise les n variables les plus importantes à partir d'un DataFrame d'importance.

    Parameters:
    - importance_df (pd.DataFrame): Un DataFrame contenant les colonnes spécifiées.
    - variable_col (str): Le nom de la colonne contenant les noms des variables.
    - importance_col (str): Le nom de la colonne contenant les valeurs d'importance.
    - top_n (int): Le nombre de variables les plus importantes à afficher.
    """
    # Vérifier si les colonnes existent
    if variable_col not in importance_df.columns or importance_col not in importance_df.columns:
        raise ValueError(f"Les colonnes '{variable_col}' et/ou '{importance_col}' ne sont pas présentes dans le DataFrame.")

    # Trier par importance décroissante
    top_variables = importance_df.sort_values(by=importance_col, ascending=False).head(top_n)

    # Création du graphique
    plt.figure(figsize=(10, 8))
    plt.barh(top_variables[variable_col], top_variables[importance_col], color='steelblue')
    plt.xlabel("Importance")
    plt.ylabel("Variable")
    plt.title(f"Top {top_n} variables les plus importantes")
    plt.gca().invert_yaxis()  # Afficher les plus importantes en haut
    plt.show()

In [ ]:
# Visualisation des variables les plus importantes
plot_top_n_variables(
    importance_df = importance_df_with_labels,
    variable_col='LIB_VAR',
    importance_col='Importance',
    top_n=10
)